# Clifford+RZ

You need the `pyqrack` package to run this notebook. [`vm6502q/pyqrack`](https://github.com/vm6502q/pyqrack) is a pure Python wrapper on the [`vm6502q/qrack`](https://github.com/vm6502q/qrack) quantum computer simulation framework core library. The preferred method of installation is from source code, at those GitHub repositories, but a package with default build precompiled binaries is available on [pypi](https://pypi.org/project/pyqrack/0.2.0/).

In [1]:
# For example, if your Jupyter installation uses pip:
# import sys
# !{sys.executable} -m pip install --platform=manylinux2014_x86_64 --only-binary=:all: pyqrack

In [2]:
import collections
import copy
import math
import os
import random
# import warnings

import numpy as np

import tensorcircuit as tc
import tensorcircuit.compiler.simple_compiler as tcsc

from qiskit import QuantumCircuit
from qiskit.compiler import transpile

from pyqrack import QrackSimulator

# If we disable OpenCL and set the max CPU qubits to max integer, we bypass qubit widths limits.
os.environ["QRACK_MAX_CPU_QB"]="-1"

In [3]:
def normalize_2x2_unitary(mtrx):
    det = np.linalg.det(mtrx)
    a = (mtrx[0][0] + np.conj(mtrx[1][1] / det)) / 2
    b = (mtrx[0][1] + np.conj(-mtrx[1][0] / det)) / 2
    return np.array([[a, b], [-det * np.conj(b), det * np.conj(a)]], dtype=np.complex128)

In [4]:
width = 4
max_magic = 4

[`QrackSimulator`](https://github.com/vm6502q/pyqrack/blob/main/pyqrack/qrack_simulator.py) is the "workhorse" of the `pyqrack` package. It instantiates simulated "registers" of qubits that we can act basic quantum gates between, to form arbitrary universal quantum circuits.

`QrackCircuit` is an optional class for optimizing compilation. With it, one can define a circuit in advance, which is optimized upon definition. Then, one can save the optimized result to a file and later load it into a new or existing `QrackCircuit` instance. Ultimately, the circuit is executed by calling `run()` on a `QrackCircuit`, with a parameter of `QrackSimulator` of appropriate size. (The necessary width of the `QrackSimulator` can be determined with `QrackCircuit.get_qubit_count()`.)

In [5]:
def rand_1qb(sim, q):
    # th = random.uniform(0, 4 * math.pi)
    ph = random.uniform(0, 4 * math.pi)
    # lm = random.uniform(0, 4 * math.pi)
    # sim.u(th, ph, lm, q)
    sim.u(0, ph, 0, q)

def random_circuit(width, circ):
    t_count = 0
    single_bit_gates = circ.h, circ.x, circ.y
    two_bit_gates = circ.cx, circ.cy, circ.cz, circ.swap, circ.iswap

    for n in range(3 * width):
        # Single bit gates
        for j in range(width):
            random.choice(single_bit_gates)(j)
            if (t_count < max_magic) and ((width * width * random.random() / max_magic) < 1):
                rand_1qb(circ, j)
                t_count += 1

        # Multi bit gates
        bit_set = [i for i in range(width)]
        while len(bit_set) > 1:
            b1 = random.choice(bit_set)
            bit_set.remove(b1)
            b2 = random.choice(bit_set)
            bit_set.remove(b2)
            random.choice(two_bit_gates)(b1, b2)

    return circ

If we have Qiskit and numpy installed, we can convert to a Qiskit circuit. The gate count and qubit width might be higher, but the circuit is entirely Clifford group, except for a single terminal layer of non-Clifford gates, (followed by either post selection or a rudimentary error-correction routine).

In [6]:
orig_circ = random_circuit(width, QuantumCircuit(width))
qsim = QrackSimulator(width, isSchmidtDecomposeMulti=False, isSchmidtDecompose=False, isOpenCL=False)
qsim.run_qiskit_circuit(orig_circ, 0)
qsim.out_to_file("qrack_circuit.chp")
qsim.reset_all()
qsim = None

In [7]:
circ = QrackSimulator.file_to_qiskit_circuit("qrack_circuit.chp")

if circ.width() <= 16:
    print(circ)
print("(Near-Clifford) gate count: ", sum(dict(circ.count_ops()).values()))
print("(Near-Clifford) depth of critical path: ", circ.depth())

          ┌───┐                               ┌───┐┌───┐     ┌───┐   ┌───┐   »
q_0: ─────┤ S ├───────X───────────────────────┤ X ├┤ H ├──■──┤ S ├───┤ H ├───»
          ├───┤       │  ┌───┐   ┌───┐        └─┬─┘└───┘  │  └───┘   └───┘   »
q_1: ─────┤ H ├───────┼──┤ X ├───┤ H ├──────────┼─────────┼──────────────────»
          └───┘┌───┐  │  └─┬─┘   └───┘   ┌───┐  │       ┌─┴─┐┌───┐┌─────────┐»
q_2: ───────■──┤ H ├──X────■───────■─────┤ X ├──■───────┤ X ├┤ X ├┤ Unitary ├»
     ┌───┐  │  ├───┤             ┌─┴─┐   └─┬─┘┌───┐     └───┘└───┘└─────────┘»
q_3: ┤ S ├──┼──┤ H ├──■──────────┤ X ├─────┼──┤ H ├──────────────────────────»
     └───┘┌─┴─┐└───┘┌─┴─┐┌───┐┌──┴───┴──┐  │  └───┘                          »
q_4: ─────┤ X ├─────┤ X ├┤ X ├┤ Unitary ├──┼─────────────────────────────────»
          ├───┤     └───┘└───┘└─────────┘  │  ┌───┐                          »
q_5: ─────┤ H ├────────────────────────────■──┤ H ├──────────────────────────»
          └───┘                               └───┘ 

Some phase gates tend to occur at the beginning of the circuit: these can safely be ignored. In fact, we can trace out the evolution of separable single-qubit states from the beginning of the circuit until they become entangled, for more gate elimination. Deterministic control qubits can be eliminated, as well.

In [8]:
def qunit_step(i, circ, to_cut, state, basis, has_depth):
    op = circ.data[i].operation
    qubits = circ.data[i].qubits
    first_qubit = circ.find_bit(qubits[0])[0]
    if has_depth[first_qubit]:
        for q in qubits[1:]:
            has_depth[circ.find_bit(q)[0]] = True
        return
    if len(qubits) == 1:
        # The overall algorithm might be very sensitive to floating-point error, so check exact equality:
        is_unitary_phase = ((op.name == "unitary") and (op.params[0][0][1] == 0) and (op.params[0][1][0] == 0))
        if op.name == "h":
            basis[first_qubit] = not basis[first_qubit]
        elif ((not basis[first_qubit]) and (is_unitary_phase or (op.name == "z") or (op.name == "s") or (op.name == "sdg"))) or (basis[first_qubit] and (op.name == "x")):
            to_cut.append(i)
        elif ((not basis[first_qubit]) and ((op.name == "x") or (op.name == "y"))) or (basis[first_qubit] and ((op.name == "y") or (op.name == "z"))):
            state[first_qubit] = not state[first_qubit]
        else:
            has_depth[first_qubit] = True
    elif op.name == "swap":
        second_qubit = circ.find_bit(qubits[1])[0]
        has_depth[first_qubit], has_depth[second_qubit] = has_depth[second_qubit], has_depth[first_qubit]
        state[first_qubit], state[second_qubit] = state[second_qubit], state[first_qubit]
        basis[first_qubit], basis[second_qubit] = basis[second_qubit], basis[first_qubit]
        if (not has_depth[first_qubit]) and (not has_depth[second_qubit]) and (state[first_qubit] == state[second_qubit]) and (basis[first_qubit] == basis[second_qubit]):
            to_cut.append(i)
    elif not basis[first_qubit] and op.name == "cx":
        if state[first_qubit]:
            c = QuantumCircuit(1)
            c.x(0)
            instr = c.data[0]
            instr.qubits = (qubits[1],)
            circ.data[i] = copy.deepcopy(instr)
            i -= 1
        else:
            to_cut.append(i)
    else:
        qc = circ.find_bit(qubits[0])[0]
        qt = circ.find_bit(qubits[1])[0]
        has_depth[qc] = True
        has_depth[qt] = True


to_cut = []
state = circ.width() * [False]
basis = circ.width() * [False]
has_depth = circ.width() * [False]
for i in range(len(circ.data)):
    qunit_step(i, circ, to_cut, state, basis, has_depth)
to_cut.reverse()
for i in to_cut:
    del circ.data[i]

to_cut = []
state = circ.width() * [False]
basis = circ.width() * [False]
has_depth = (width * [True]) + ((circ.width() - width) * [False])
for i in reversed(range(len(circ.data))):
    qunit_step(i, circ, to_cut, state, basis, has_depth)
for i in to_cut:
   del circ.data[i]

if circ.width() <= 16:
    print(circ)
print("(Near-Clifford) gate count: ", sum(dict(circ.count_ops()).values()))
print("(Near-Clifford) depth of critical path: ", circ.depth())

                    ┌───┐   ┌───┐        ┌───┐   ┌───┐   ┌───┐   ┌───┐┌───┐»
q_0: ───────X───────┤ X ├───┤ H ├─────■──┤ S ├───┤ H ├───┤ S ├─X─┤ X ├┤ H ├»
     ┌───┐  │  ┌───┐└─┬─┘   └───┘     │  └───┘   └───┘   └───┘ │ └─┬─┘└───┘»
q_1: ┤ H ├──┼──┤ H ├──┼───────────────┼────────────────────────┼───┼───────»
     ├───┤  │  ├───┤  │             ┌─┴─┐┌───┐┌─────────┐      │   │       »
q_2: ┤ H ├──X──┤ X ├──■─────────────┤ X ├┤ X ├┤ Unitary ├──────┼───┼───────»
     ├───┤     └─┬─┘┌───┐           └───┘└───┘└─────────┘      │   │  ┌───┐»
q_3: ┤ H ├──■────┼──┤ H ├──────────────────────────────────────X───■──┤ Y ├»
     └───┘┌─┴─┐  │  ├───┤┌─────────┐                                  └───┘»
q_4: ─────┤ X ├──┼──┤ X ├┤ Unitary ├───────────────────────────────────────»
     ┌───┐└───┘  │  ├───┤└─────────┘                                       »
q_5: ┤ H ├───────■──┤ H ├──────────────────────────────────────────────────»
     └───┘          └───┘                                                  »

If any ancilla qubit is nothing but an injection gadget, we can re-inject it.

In [9]:
sqrt1_2 = 1 / math.sqrt(2)
passable_gates = ["unitary", "h", "x", "y", "z", "s", "sdg"]

for i in range(width, circ.width()):
    # We might trace out swap, but we want to maintain the iteration order of qubit channels.
    non_clifford = np.array([[1, 0], [0, 1]], np.complex128)
    j = len(circ.data) - 1
    while j >= 0:
        op = circ.data[j].operation
        qubits = circ.data[j].qubits
        id = circ.find_bit(qubits[0])[0]
        if (len(qubits) < 2) and (id == i):
            if op.name == "unitary":
                non_clifford = np.matmul(non_clifford, op.params[0])
            elif op.name == "h":
                non_clifford = np.matmul(non_clifford, np.array([[sqrt1_2, sqrt1_2], [sqrt1_2, -sqrt1_2]], np.complex128))
            elif op.name == "x":
                non_clifford = np.matmul(non_clifford, np.array([[0, 1], [1, 0]], np.complex128))
            elif op.name == "y":
                non_clifford = np.matmul(non_clifford, np.array([[0, -1j], [1j, 0]], np.complex128))
            elif op.name == "z":
                non_clifford = np.matmul(non_clifford, np.array([[1, 0], [0, -1]], np.complex128))
            elif op.name == "s":
                non_clifford = np.matmul(non_clifford, np.array([[1, 0], [0, 1j]], np.complex128))
            elif op.name == "sdg":
                non_clifford = np.matmul(non_clifford, np.array([[1, 0], [0, -1j]], np.complex128))
            else: 
                print("Something went wrong!")

            del circ.data[j]
            j -= 1
            continue

        if len(qubits) < 2:
            j -= 1
            continue

        if op.name == "swap":
            partner = circ.find_bit(qubits[1])[0]
            if i == id:
                i = partner
            if i == partner:
                i = id
            j -= 1
            continue

        # The overall algorithm might be very sensitive to floating-point error, so check exact equality:
        is_buffer_identity = np.array_equal(non_clifford, np.eye(2, dtype=np.complex128))

        # if (id == i) and (op.name == "cx" or op.name == "cy" or op.name == "cz"):
            # Either way, we're cutting this gate.
            # orig_instr = circ.data[j]
            # del circ.data[j]
            
            # if is_buffer_identity:
                # If we're not buffering anything but post selection, the blocking gate has no effect, and we're safe to continue.
            #     j -= 1
            #     continue

            # h = QuantumCircuit(1)
            # h.h(0)
            # instr = h.data[0]

            # We're replaced CNOT with CNOT in the opposite direction plus four H gates
            # instr.qubits = (qubits[0],)
            # circ.data.insert(j, copy.deepcopy(instr))
            # instr.qubits = (qubits[1],)
            # circ.data.insert(j, copy.deepcopy(instr))
            # orig_instr.qubits = (qubits[1], qubits[0])
            # circ.data.insert(j, copy.deepcopy(orig_instr))
            # instr.qubits = (qubits[0],)
            # circ.data.insert(j, copy.deepcopy(instr))
            # instr.qubits = (qubits[1],)
            # circ.data.insert(j, copy.deepcopy(instr))
            # j += 4

            # continue

        if (id == i) or (op.name != "cx"):
            if is_buffer_identity:
                # No buffer content to write to circuit definition
                break

            # We're blocked, so we insert our buffer at this place in the circuit definition.
            c = QuantumCircuit(1)
            c.unitary(non_clifford, 0)
            instr = c.data[0]
            instr.qubits = (qubits[0],)
            circ.data.insert(j + 1, copy.deepcopy(instr))
            break

        if circ.find_bit(qubits[1])[0] == i:
            to_inject = np.matmul(non_clifford, np.array([[sqrt1_2, sqrt1_2], [sqrt1_2, -sqrt1_2]]))
            if np.array_equal(to_inject, np.eye(2, dtype=np.complex128)):
                # No buffer content to write to circuit definition
                del circ.data[j]
                j -= 1
                continue

            c = QuantumCircuit(1)
            c.unitary(to_inject, 0)
            instr = c.data[0]
            instr.qubits = (qubits[0],)
            circ.data[j] = copy.deepcopy(instr)

        j -= 1

    if (j < 0) and not np.array_equal(non_clifford, np.eye(2, dtype=np.complex128)):
        c = QuantumCircuit(1)
        c.unitary(non_clifford, 0)
        instr = c.data[0]
        instr.qubits = (qubits[0],)
        circ.data.insert(0, copy.deepcopy(instr))

if circ.width() <= 16:
    print(circ)
print("(Near-Clifford) gate count: ", sum(dict(circ.count_ops()).values()))
print("(Near-Clifford) depth of critical path: ", circ.depth())

                                     ┌───┐┌───┐     ┌───┐   ┌───┐   ┌───┐   »
q_0: ─────────────X──────────────────┤ X ├┤ H ├──■──┤ S ├───┤ H ├───┤ S ├─X─»
        ┌───┐     │     ┌───┐        └─┬─┘└───┘  │  └───┘   └───┘   └───┘ │ »
q_1: ───┤ H ├─────┼─────┤ H ├──────────┼─────────┼────────────────────────┼─»
        ├───┤     │     └───┘   ┌───┐  │       ┌─┴─┐┌───┐┌─────────┐      │ »
q_2: ───┤ H ├─────X─────────────┤ X ├──■───────┤ X ├┤ X ├┤ Unitary ├──────┼─»
     ┌──┴───┴──┐┌───┐┌─────────┐└─┬─┘┌───┐     └───┘└───┘└─────────┘      │ »
q_3: ┤ Unitary ├┤ H ├┤ Unitary ├──┼──┤ H ├────────────────────────────────X─»
     └─────────┘└───┘└─────────┘  │  └───┘                                  »
q_4: ─────────────────────────────┼─────────────────────────────────────────»
        ┌───┐                     │  ┌───┐                                  »
q_5: ───┤ H ├─────────────────────■──┤ H ├──────────────────────────────────»
        └───┘                        └───┘                      

We can further optimize by post-selecting with tensor network software.

In [10]:
# warnings.resetwarnings()
tc.set_backend("tensorflow")
tc.set_contractor("auto")
tc.set_dtype("complex64")

net = tc.Circuit.from_qiskit(circ)
for b in range(width, circ.width()):
    net.post_select(b, keep=0)
net = tcsc.simple_compile(net)[0]

We can sample with tensorcircuit at this point, or we can try another round of optimization.

In [11]:
# %%time
# an_samples = net.sample(allow_state=True, batch=1000000, format="count_dict_bin")
# sample_map = {}
# for key, value in an_samples.items():
#     k = (int(key) & ((1 << width) - 1))
#     sample_map[k] =  (sample_map[k] + value) if k in sample_map else value
# samples = []
# for key, value in sample_map.items():
#     for i in range(value):
#         samples.append(key)

We can convert back to a Qiskit circuit. (Should any warnings appear, this step might produce a logically-incorrect circuit.)

In [12]:
# warnings.filterwarnings("error")
# try:
#     _circ = net.to_qiskit()
#     circ = _circ
#     if circ.width() <= 16:
#         print(circ)
#     print("(Near-Clifford) gate count: ", sum(dict(circ.count_ops()).values()))
#     print("(Near-Clifford) depth of critical path: ", circ.depth())
# except:
#     print("Failed to convert from tensorcircuit to Qiskit circuit.")

`QrackSimulator` can run a Qiskit circuit. (After the file is saved and loaded, separable ancillae qubits will be removed.)

In [13]:
basis_gates=["rz", "h", "x", "y", "z", "sx", "sy", "s", "sdg", "cx", "cy", "cz", "swap", "iswap", "iswap_dg"]
circ = transpile(circ, basis_gates=basis_gates, optimization_level=3)

os.rename("qrack_circuit.chp", "qrack_circuit_original.chp")

qsim = QrackSimulator(circ.width(), isSchmidtDecomposeMulti=False, isSchmidtDecompose=False, isOpenCL=False)
qsim.run_qiskit_circuit(circ, 0)

qsim.out_to_file("qrack_circuit.chp")

lines = []
with open("qrack_circuit.chp","r",encoding="utf-8") as file:
    lines = file.readlines()
lines[0] = str(width) + "\n"
with open("qrack_circuit.chp", "w", encoding="utf-8") as file:
    file.writelines(lines)

qsim = QrackSimulator.in_from_file("qrack_circuit.chp")
qsim.out_to_file("qrack_circuit.chp")

We can also sample with Qrack, as opposed to tensor networks.

In [14]:
lines = []
with open("qrack_circuit.chp","r",encoding="utf-8") as file:
    lines = file.readlines()
chp_qb_count = int(lines[1])
is_clifford = True
for i in range(2 + 2 * chp_qb_count, len(lines)):
    if lines[i] != "(1,0) (0,0) (0,0) (1,0)\n":
        is_clifford = False
        break

samples = []
if is_clifford:
    qsim = QrackSimulator.in_from_file("qrack_circuit.chp")

    %time
    print(qsim.measure_shots(list(range(width)), 256))

In [15]:
def fidelities_from_measurement_results(measurement_list, qiskit_circuit):
    perm_count = 1 << qiskit_circuit.width()
    ideal_shots = perm_count << 12

    sim = QrackSimulator(qubitCount = qiskit_circuit.width(), isStabilizerHybrid=False, isOpenCL=False)
    sim.run_qiskit_circuit(qiskit_circuit, 0)
    ideal_result = sim.measure_shots(list(range(qiskit_circuit.width())), ideal_shots)
    ideal_result = dict(collections.Counter(ideal_result))
    for key, value in ideal_result.items():
        ideal_result[key] = value / ideal_shots
        
    ideal_probs_sv = []
    for i in range(perm_count):
        ideal_probs_sv.append(ideal_result[i] if i in ideal_result else 0)
    
    ideal_probs = []
    for bit_string in measurement_list:
        ideal_probs.append(ideal_probs_sv[bit_string])
                               
    return (perm_count * np.mean(ideal_probs)) - 1

if circ.width() <= 24:
    # The circuit has 0 ancillae, so we can generate samples with conventional simulation.
    qsim = QrackSimulator(qubitCount = circ.width(), isStabilizerHybrid=False, isOpenCL=False)
    qsim.run_qiskit_circuit(circ, 0)
    for i in range(width, circ.width()):
        qsim.force_m(i, False)
    samples = qsim.measure_shots(list(range(width)), 1000000)
    print("XEB fidelity: ", fidelities_from_measurement_results(samples, orig_circ))

XEB fidelity:  0.3968374711914062
